In [1]:
import os 
# since notebook wd is /home/pistachio/pistachio/research/trials.ipynb, changning 
# wd to E:\Pistachio_Classification_Project
%pwd
from pathlib import Path
os.chdir(Path("E:/Pistachio_Classification_Project"))
%pwd

'E:\\Pistachio_Classification_Project'

### 1. edit config.yaml
prepare_callbacks:
    root_dir: artifacts/prepare_callbacks
    tensorboard_root_log_dir: artifacts/prepare_callbacks/tensorboard_root_log_dir
    checkpoint_model_filepath: artifacts/prepare_callbacks/checkpoint_dir/model.h5


### 3.update the entity 

In [2]:
from dataclasses import dataclass
from pathlib import Path 
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path 
    tensorboard_root_log_dir:Path
    checkpoint_model_file_path: Path
    patience: int
    monitor: str 

### 4. Updating the configuration Manager in src/config/configuration.py

In [3]:
from cnnClassifier.constants.__init__ import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
from pathlib import Path
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config= read_yaml(config_file_path) # return ConfigBox object with 'artifacts_root' as a key name
        self.params= read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def prepare_callbacks(self)->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        # create directories
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_checkpoint_dir), Path(config.tensorboard_root_log_dir)])
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),  
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),  
            checkpoint_model_file_path = Path(config.checkpoint_model_filepath), 
            patience= self.params.PATIENCE, 
            monitor= self.params.MONITOR)  

        return prepare_callbacks_config

### 5. Updating the components

In [4]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time 

In [5]:
class PrepareCallbacks:
    def __init__(self,  config: PrepareCallbacksConfig):
        self.config = config
    
    def model_checkpoint_callbacks(self)-> tf.keras.callbacks.ModelCheckpoint:
        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_file_path,
            monitor='val_loss',
            verbose=1,
            save_best_only=True,
            save_weights_only=True,
            mode='auto',
            save_freq="epoch")
        
        return model_checkpoint

    def early_stopping_callbacks(self)-> tf.keras.callbacks.EarlyStopping:
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0,
            patience=self.config.patience,
            verbose=1,
            mode='auto',
            restore_best_weights=True)
        
        return early_stopping
    
    def tensorboard_callbacks(self)-> tf.keras.callbacks.TensorBoard:
        log_dir = str(Path(self.config.tensorboard_root_log_dir))
        tb_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_images=True)
        return tb_callbacks
    
    
    

### 6. Update the pipeline

In [6]:
try:
    conf_manager = ConfigurationManager()
    prepare_callbacks = conf_manager.prepare_callbacks()
    callbacks = PrepareCallbacks(prepare_callbacks)
    callbacks_cp = callbacks.model_checkpoint_callbacks
    callbacks_es = callbacks.early_stopping_callbacks
    callbacks_tb = callbacks.early_stopping_callbacks
except Exception as e:
    raise e

[2023-11-03 10:55:12,325: INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-11-03 10:55:12,328: INFO : common : yaml file: params.yaml loaded successfully]
[2023-11-03 10:55:12,329: INFO : common : created directory at: artifacts]
[2023-11-03 10:55:12,330: INFO : common : created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-11-03 10:55:12,331: INFO : common : created directory at: artifacts\prepare_callbacks\tensorboard_root_log_dir]


### Final Step -> Convert everything into modular coding
1. we have already updated the config/config.yaml file for the creating base cnn model
2. we also have updated the params.yaml file 
3. now update the enity  (copy entity code blocks to src/entity/config_enity.py)
4. update configuration manager in src config(copy ConfigManager code blocks to src/config/configuration.py)
5. update the components (copy component code blocks to src/components/createnewfile(base_model.py))
6. update the pipeline(copy pipeline code blocks to src/pipeine/stage_02_base_model.py)
7. update the main file